# Deep Learning from Udacity (Google)
## Assignment 1
Use notMNIST dataset to classify numbers
<br>For starters, we will import the python libraries that we need

In [1]:
\c 200 200
plt:.p.import[`matplotlib.pyplot];
np:.p.import[`numpy];
os:.p.import[`os];
sys:.p.import[`sys];
tarfile:.p.import[`tarfile];
display:.p.import[`IPython.display][`:display];
Image:.p.import[`IPython.display][`:Image];
skimage:.p.import[`skimage];
imageio:.p.import[`imageio];

### Download functionality
After importing, we have to declare local python callbacks that we need.
<br> In this example, we need a download progress hook to monitor the download progress, this is not necessary but useful to show that the function is running
<br>
<br> The hook is declared in python while the download functionality is embedpy

In [2]:
/%declare download hook to monitor download progress
p)last_percent_reported=None
p)def download_progress_hook(count, blockSize, totalSize):
  import sys
  global last_percent_reported

  percent = int(count * blockSize * 100 / totalSize)
 
  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent

download_progress_hook:.p.get[`download_progress_hook]

In [3]:
url:"http://yaroslavvb.com/upload/notMNIST/"
dataStore:`:DeepLearning
if[()~key dataStore;
    -1"Deep Learning Folder is missing, creating it now";
    system "mkdir DeepLearning"]
notMNISTLarge:`notMNIST_large.tar.gz
notMNISTSmall:`notMNIST_small.tar.gz

downloadFile:{[file;dateStore;url] 
    urllib:.p.import[`urllib][`:request];
    fileDir:` sv dataStore,file;
    url:url,string file;
    -1"Check if ",1_string[fileDir]," exists?";
    $[()~key fileDir;
        [-1"File is missing, initialise download";
        urllib[`:urlretrieve][url;1_string fileDir;`reporthook pykw download_progress_hook];
        /0N! 1_string fileDir;
        /httpGet[]
        /filename:urlretrieve[url;1_string fileDir][];
        -1"\nDownload Complete!";];
        -1"File is present, skip download";
        ]
    }



In [4]:
downloadFile[notMNISTLarge;dataStore;url];
-1"\n";
downloadFile[notMNISTSmall;dataStore;url];

Check if DeepLearning/notMNIST_large.tar.gz exists?
File is present, skip download


Check if DeepLearning/notMNIST_small.tar.gz exists?
File is present, skip download


### Untaring of data
After the files are downloaded into the directory we want them in, we will have to untar the data
<br> Note that the directory structure is known before hand.
<br> If it was not known, we will have toe use recursive key to know its structure dynamically.

In [5]:
/Note This can be done with system tar -xzf in linux

unTarFile:{[file;dateStore]
    fileNoTar: `$first "." vs string file;
    fileDir:` sv dataStore,file;
    folderDir: 1_(first "." vs string fileDir),"/.";
    -1"Check if ",1_string[fileDir]," exists?";
    if[10=count childDir:key parentDir:` sv (folderDirQ:hsym `$-2_folderDir),fileNoTar;
        -1"Extracted Files Exists! \nSkipping Extraction for ",string file;
        :` sv' parentDir,/:childDir];
    if[file in key dateStore;
        -1"File exists, proceed to untar into ",-1_folderDir;
        -1"This will take a while, please be patient";
        tar:tarfile[`:open][1_string fileDir];
        tar[`:extractall][`path pykw folderDir];
        tar[`:close][];
        ];
    if[fileNoTar in key folderDirQ;
       $[10=count childDir:key parentDir;
            [-1"Extraction Completed! \nAll 10 Folders are Present";` sv' parentDir,/:childDir];
            -2"Extraction Failed! \nThere Are Missing Folders";
            ]
        ]
    
    }


In [6]:
notMNISTSmallFolders:unTarFile[notMNISTSmall;dataStore]
-1"\n";
notMNISTLargeFolders:unTarFile[notMNISTLarge;dataStore]

Check if DeepLearning/notMNIST_small.tar.gz exists?
Extracted Files Exists! 
Skipping Extraction for notMNIST_small.tar.gz


Check if DeepLearning/notMNIST_large.tar.gz exists?
Extracted Files Exists! 
Skipping Extraction for notMNIST_large.tar.gz


## Visualisation of Downloaded Data
We might want to randomly look at some of the image files which we have downloaded

In [7]:
3#notMNISTSmallFolders
3#notMNISTLargeFolders

randomSelectionA:1_'string ` sv' notMNISTSmallFolders[0],/:2?key notMNISTSmallFolders 0
randomSelectionB:1_'string ` sv' notMNISTSmallFolders[1],/:2?key notMNISTSmallFolders 1

display each Image each randomSelectionA;
display each Image each randomSelectionB;

`:DeepLearning/notMNIST_small/notMNIST_small/A`:DeepLearning/notMNIST_small/notMNIST_small/B`:DeepLearning/notMNIST_small/notMNIST_small/C


`:DeepLearning/notMNIST_large/notMNIST_large/A`:DeepLearning/notMNIST_large/notMNIST_large/B`:DeepLearning/notMNIST_large/notMNIST_large/C


## Load Image into DataSet

Each of the folder represent a specific letter with its image inside
<br>Each image is 28 X 28 in size, this represents 784 pixels with each image
<br>We would have to store each of this 784 pixels into a numerical dataset which we will be able to work on with tensorFlow later

In [8]:
/here we try to load the small image dataset with the help scikit image
.utils.path:{$[x~k:key x;x;11h=type k;raze (.z.s ` sv x,)each k;()]}
/note this func is highly recursive, do not perform on root, 
/this is useful is there is weird nesting but slow if we know the structure.
notMINSTLargeNest:`:DeepLearning/notMNIST_large/notMNIST_large
notMINSTLargeNest:notMINSTAlphabet!(` sv notMINSTLargeNest,) each notMINSTAlphabet:key notMINSTLargeNest
/this contains the nested directory of each alphabet
/going forward we need to load all the for each alphabet into a q table
3#notMINSTLargeNest

A| :DeepLearning/notMNIST_large/notMNIST_large/A
B| :DeepLearning/notMNIST_large/notMNIST_large/B
C| :DeepLearning/notMNIST_large/notMNIST_large/C


In [24]:
/load directories
/lazy implemetation of utils.path
loadDir:{
    {
        @[{value;-1"Already contains paths ",string x;};
        x;
        {x set .utils.path[(value -1_string x)`$-1#string x];-1"Stored ",string x;y}x];
    }each `$"notMINSTLargeNest",/:10#.Q.A
 }
loadDir`;

(2#value @) each `$"notMINSTLargeNest",/:10#.Q.A


Already contains paths notMINSTLargeNestA
Already contains paths notMINSTLargeNestB
Already contains paths notMINSTLargeNestC
Already contains paths notMINSTLargeNestD
Already contains paths notMINSTLargeNestE
Already contains paths notMINSTLargeNestF
Already contains paths notMINSTLargeNestG
Already contains paths notMINSTLargeNestH
Already contains paths notMINSTLargeNestI
Already contains paths notMINSTLargeNestJ


:DeepLearning/notMNIST_large/notMNIST_large/A/ISBKYW1pcm9xdWFpICEudHRm.png :DeepLearning/notMNIST_large/notMNIST_large/A/IVNrZXRjaHkgVGltZXMgQm9sZC50dGY=.png
:DeepLearning/notMNIST_large/notMNIST_large/B/ISBKYW1pcm9xdWFpICEudHRm.png :DeepLearning/notMNIST_large/notMNIST_large/B/IVNrZXRjaHkgVGltZXMgQm9sZC50dGY=.png
:DeepLearning/notMNIST_large/notMNIST_large/C/ISBKYW1pcm9xdWFpICEudHRm.png :DeepLearning/notMNIST_large/notMNIST_large/C/IVNrZXRjaHkgVGltZXMgQm9sZC50dGY=.png
:DeepLearning/notMNIST_large/notMNIST_large/D/ISBKYW1pcm9xdWFpICEudHRm.png :DeepLearning/notMNIST_large/notMNIST_large/D/IVNrZXRjaHkgVGltZXMgQm9sZC50dGY=.png
:DeepLearning/notMNIST_large/notMNIST_large/E/ISBKYW1pcm9xdWFpICEudHRm.png :DeepLearning/notMNIST_large/notMNIST_large/E/IVNrZXRjaHkgVGltZXMgQm9sZC50dGY=.png
:DeepLearning/notMNIST_large/notMNIST_large/F/ISBKYW1pcm9xdWFpICEudHRm.png :DeepLearning/notMNIST_large/notMNIST_large/F/IVNrZXRjaHkgVGltZXMgQm9sZC50dGY=.png
:DeepLearning/notMNIST_large/notMNIST_large/G/ISBKYW

In [28]:
/create table to upsert data into
(`$"largeTable",/:10#.Q.A) set\: flip `alphabet`imgData!(();())
largeTableA

`largeTableA`largeTableB`largeTableC`largeTableD`largeTableE`largeTableF`largeTableG`largeTableH`largeTableI`largeTableJ


alphabet imgData
----------------


In [29]:
/define func to read in each image and store them as q table
imageIngestion:{[imgPath;t;alpha]
    /read in img with imageio
    @[{imgData:raze `float$imageio[`:imread][1_string x 0]`;
        /validation to ensure that image is 28x28
        $[784=count imgData;
            x[1] upsert (x 2;imgData);
            -1"imgPath ",string[x 0]," skipped"];
        };
        (imgPath;t;alpha);
        {-1"imgPath ",1_string[x]," couldn't be read, skipped ",.Q.s y}imgPath];    
    }


In [68]:
/note that this is a very expensive operation and can take up to 30 mins for the entire ingestion
/we will save the table down as a q table to make it efficient
/if table has been saved down, it will be skipped
ingestFunc:{[imgPath]
    alpha:last ` vs first ` vs first imgPath;
    t:`$"largeTable",string alpha;
    $[()~key toStore:` sv `:DeepLearning/notMNIST_large/flatTable,t;
     [-1"Start of Image Ingestion ",string t;
        imageIngestion[;t;alpha] each imgPath;
        -1"Saving Down Flat File to HDB";
        toStore set value t;
        ];
    -1"Flat table exist, skipping ingestion for ",string t];    
    } 

/running the ingestion function
(ingestFunc value @) each "notMINSTLargeNest",/:10#.Q.A;



Start of Image Ingestion largeTableA
imgPath DeepLearning/notMNIST_large/notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png couldn't be read, skipped "call: Could not find a format to read the specified file in mode 'i'"

imgPath DeepLearning/notMNIST_large/notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png couldn't be read, skipped "call: Could not find a format to read the specified file in mode 'i'"

imgPath DeepLearning/notMNIST_large/notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png couldn't be read, skipped "call: Could not find a format to read the specified file in mode 'i'"

Saving Down Flat File to HDB
Start of Image Ingestion largeTableB
imgPath DeepLearning/notMNIST_large/notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png couldn't be read, skipped "call: Could not find a format to read the specified file in mode 'i'"

Saving Down Flat File to HDB
Start of Image Ingestion largeTableC
Saving Down Flat File to HDB
Flat table exist, skipping ingestion for largeTabl

In [79]:
-1"Following Tables are in local drive: ",.Q.s keptTable:keptTable where (keptTable:`#key `:DeepLearning/notMNIST_large/flatTable)like "*largeTable*";
(`$'10#.Q.A)!(count get @)each ` sv' `:DeepLearning/notMNIST_large/flatTable,/:keptTable

Following Tables are in local drive: `largeTableA`largeTableB`largeTableC`largeTableD`largeTableE`largeTableF`largeTableG`largeTableH`largeTableI`largeTableJ



A| 52909
B| 52911
C| 52912
D| 105822
E| 52912
F| 52912
G| 52912
H| 52912
I| 52912
J| 52911


## Normalisation of Data
After the long and challenging process to ingest all the image data, now we have proper q hdb table with the image arrays to work with.

To speed up machine learning process, we will adopt normalisation of data which is to subtract the mean followed by the std deviation.

There are other methods of data manipulation which includes scaling to (0,1) but normalisation is chosen in this study.

In [ ]:
/get table and normalise the data

/attempt to save down to disk